In [24]:
# Numeric
import numpy as np
import pandas as pd

# Tools
import os, sys
import re
import string
import csv
import string
import datetime
from datetime import datetime

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.ticker as mticker
import matplotlib.pyplot as plt

import src.utils_variable as utils
%matplotlib inline

In [25]:
df = pd.read_csv(utils.cloudCoveragecsv)
df = df[ (df["folderPath"].str.contains("smoke") == False) & \
         (df["folderPath"].str.contains("cloudy/") == False) & \
         (df["folderPath"].str.contains("cloudyClean/") == False)
        ]
df_clean = df.drop(columns= ["folderPath", "startDate", "endDate"])
df_clean.head()

,EMSR_AOI,sizeImage,burnedPixel,cloudPixel,countOverlap,percentageCloud,percentageOverlap,Type
0,EMSR443_AOI01_01,2883060,229351,0,0,0.00,0.0,GRA
1,EMSR443_AOI01_01,2883060,207100,0,0,0.00,0.0,DEL
2,EMSR632_AOI01_01,1518048,529058,23383,4767,1.54,0.9,GRA
3,EMSR632_AOI01_01,1518048,375227,23383,0,1.54,0.0,DEL
4,EMSR453_AOI01_01,1167774,159042,0,0,0.00,0.0,GRA


In [26]:
df_GRA = df_clean[ df_clean["Type"] == "GRA"]
df_DEL = df_clean[ df_clean["Type"] == "DEL"]
df_FEP = df_clean[ df_clean["Type"] == "FEP"]

In [27]:
print(len(df_GRA))
print(len(df_DEL))
print(len(df_FEP))

233
186
125


In [28]:
#GRA
df_GRA_Over = df_GRA.groupby(by=["EMSR_AOI"], sort=True, as_index=False)["countOverlap"].sum() 
df_GRA_Burn = df_GRA.groupby(by=["EMSR_AOI"], sort=True, as_index=False)["burnedPixel"].sum()
df_GRA_size = df_GRA.groupby(by=["EMSR_AOI"], sort=True, as_index=False)["sizeImage"].sum()

# df_GRA_percentage = pd.merge(df_GRA_Burn, df_GRA_Over, on=["EMSR","AOI"])
df_GRA_percentage = df_GRA_size.merge(df_GRA_Burn, on=["EMSR_AOI"]).merge(df_GRA_Over, on=["EMSR_AOI"])
df_GRA_percentage["percentage Covered"] = df_GRA_percentage["countOverlap"]/df_GRA_percentage["burnedPixel"]*100
df_GRA_percentage = df_GRA_percentage.sort_values("percentage Covered", ascending=False)
df_GRA_percentage[:30]

,EMSR_AOI,sizeImage,burnedPixel,countOverlap,percentage Covered
140,EMSR533_AOI02_04,1951641,147322,4961,3.367454
228,EMSR632_AOI01_01,1518048,529058,4767,0.901035
207,EMSR599_AOI02_01,2685232,271395,2158,0.795151
152,EMSR560_AOI01_01,561120,29684,212,0.714189
136,EMSR533_AOI01_15,3862378,761611,5247,0.688934
69,EMSR458_AOI01_01,2551594,334378,2264,0.677078
130,EMSR533_AOI01_09,3860864,110155,680,0.617312
138,EMSR533_AOI02_02,1946795,400342,2374,0.592993
139,EMSR533_AOI02_03,1952090,270632,1004,0.370983
109,EMSR527_AOI05_01,2345480,887137,1266,0.142706


In [29]:
#DEL
df_DEL_Over = df_DEL.groupby(by=["EMSR_AOI"], sort=True, as_index=False)["countOverlap"].sum() 
df_DEL_Burn = df_DEL.groupby(by=["EMSR_AOI"], sort=True, as_index=False)["burnedPixel"].sum()
df_DEL_size = df_DEL.groupby(by=["EMSR_AOI"], sort=True, as_index=False)["sizeImage"].sum()

# df_DEL_percentage = pd.merge(df_DEL_Burn, df_DEL_Over, on=["EMSR","AOI"])
df_DEL_percentage = df_DEL_size.merge(df_DEL_Burn, on=["EMSR_AOI"]).merge(df_DEL_Over, on=["EMSR_AOI"])
df_DEL_percentage["percentage Covered"] = df_DEL_percentage["countOverlap"]/df_DEL_percentage["burnedPixel"]*100
df_DEL_percentage = df_DEL_percentage.sort_values("percentage Covered", ascending=False)
df_DEL_percentage[:20]

,EMSR_AOI,sizeImage,burnedPixel,countOverlap,percentage Covered
102,EMSR535_AOI02_01,1754346,77973,7742,9.929078
99,EMSR533_AOI02_04,1951641,107084,4723,4.410556
166,EMSR611_AOI01_01,801780,39086,1670,4.272630
95,EMSR533_AOI01_15,3862378,470266,5247,1.115752
97,EMSR533_AOI02_02,1946795,249168,2361,0.947553
159,EMSR599_AOI02_01,2685232,270341,2116,0.782715
33,EMSR458_AOI01_01,2551594,255623,1916,0.749541
98,EMSR533_AOI02_03,1952090,147898,854,0.577425
120,EMSR560_AOI01_01,561120,44989,224,0.497899
170,EMSR619_AOI01_03,1490397,556978,1766,0.317068


In [30]:
#FEP
df_FEP_Over = df_FEP.groupby(by=["EMSR_AOI"], sort=True, as_index=False)["countOverlap"].sum() 
df_FEP_Burn = df_FEP.groupby(by=["EMSR_AOI"], sort=True, as_index=False)["burnedPixel"].sum()
df_FEP_size = df_FEP.groupby(by=["EMSR_AOI"], sort=True, as_index=False)["sizeImage"].sum()

# df_FEP_percentage = pd.merge(df_FEP_Burn, df_FEP_Over, on=["EMSR_AOI"])
df_FEP_percentage = df_FEP_size.merge(df_FEP_Burn, on=["EMSR_AOI"]).merge(df_FEP_Over, on=["EMSR_AOI"])
df_FEP_percentage["percentage Covered"] = df_FEP_percentage["countOverlap"]/df_FEP_percentage["burnedPixel"]*100
df_FEP_percentage = df_FEP_percentage.sort_values("percentage Covered", ascending=False)
df_FEP_percentage[:20]

,EMSR_AOI,sizeImage,burnedPixel,countOverlap,percentage Covered
4,EMSR383_AOI01_04,3613800,64057,2188,3.415708
82,EMSR383_AOI04_17,3932064,103399,2766,2.675074
36,EMSR383_AOI01_36,3611880,162200,3773,2.326141
62,EMSR383_AOI03_26,4015872,147605,1430,0.968802
112,EMSR383_AOI04_47,3918330,536182,4401,0.820803
74,EMSR383_AOI04_09,3928050,1584220,12861,0.811819
63,EMSR383_AOI03_27,4017864,178052,807,0.453238
106,EMSR383_AOI04_41,3922248,598252,2011,0.336146
15,EMSR383_AOI01_15,3602400,889948,1742,0.195742
64,EMSR383_AOI03_28,4019856,178329,341,0.191220


In [31]:
# Images with most burned area covered by cloud in the single image
cloudyBurnedImages = df[["EMSR_AOI", "percentageOverlap"]].sort_values("percentageOverlap", ascending=False)
cloudyBurnedImages[:30]

,EMSR_AOI,percentageOverlap
327,EMSR535_AOI02_01,9.93
53,EMSR533_AOI02_04,4.41
60,EMSR611_AOI01_01,4.27
966,EMSR383_AOI01_04,3.42
52,EMSR533_AOI02_04,3.37
1013,EMSR383_AOI04_17,2.68
956,EMSR383_AOI01_36,2.33
35,EMSR533_AOI01_15,1.12
989,EMSR383_AOI03_26,0.97
49,EMSR533_AOI02_02,0.95


In [32]:
#Images with most cloud presence in the single image
cloudyImages = df[["EMSR_AOI", "percentageCloud", "percentageOverlap"]].sort_values("percentageCloud", ascending=False)
cloudyImages[:50]

,EMSR_AOI,percentageCloud,percentageOverlap
332,EMSR535_AOI04_03,2.02,0.10
2,EMSR632_AOI01_01,1.54,0.90
3,EMSR632_AOI01_01,1.54,0.00
128,EMSR571_AOI01_01,1.51,0.00
127,EMSR571_AOI01_01,1.51,0.00
294,EMSR560_AOI01_01,1.46,0.50
293,EMSR560_AOI01_01,1.46,0.71
1059,EMSR549_AOI01_01,1.44,0.00
327,EMSR535_AOI02_01,1.25,9.93
60,EMSR611_AOI01_01,1.21,4.27


In [33]:
#AOI cloudy covered
df_cloud = df.groupby(by=["EMSR_AOI"], sort=True, as_index=False)["cloudPixel"].sum()
df_burned = df.groupby(by=["EMSR_AOI"], sort=True, as_index=False)["burnedPixel"].sum()
df_size = df.groupby(by=["EMSR_AOI"], sort=True, as_index=False)["sizeImage"].sum()
df_overlap = df.groupby(by=["EMSR_AOI"], sort=True, as_index=False)["countOverlap"].sum()

df_percentage = df_size.merge(df_cloud, on=["EMSR_AOI"]).merge(df_overlap, on=["EMSR_AOI"]).merge(df_burned, on=["EMSR_AOI"])
df_percentage["percentage Cloudy Covered"] = df_percentage["cloudPixel"]/df_percentage["sizeImage"]*100
df_percentage["percentage Overlap Covered"] = df_percentage["countOverlap"]/df_percentage["burnedPixel"]*100
df_percentage = df_percentage.sort_values("percentage Overlap Covered", ascending=False)
df_percentage[:30]

,EMSR_AOI,sizeImage,cloudPixel,countOverlap,burnedPixel,percentage Cloudy Covered,percentage Overlap Covered
289,EMSR535_AOI02_01,1754346,21952,7742,77973,1.251293,9.929078
376,EMSR611_AOI01_01,801780,9700,1670,39086,1.209808,4.272630
286,EMSR533_AOI02_04,3903282,11172,9684,254406,0.286221,3.806514
30,EMSR383_AOI01_04,3613800,5715,2188,64057,0.158144,3.415708
108,EMSR383_AOI04_17,3932064,25601,2766,103399,0.651083,2.675074
62,EMSR383_AOI01_36,3611880,42533,3773,162200,1.177586,2.326141
88,EMSR383_AOI03_26,4015872,36152,1430,147605,0.900228,0.968802
282,EMSR533_AOI01_15,7724756,11364,10494,1231877,0.147111,0.851871
138,EMSR383_AOI04_47,3918330,11751,4401,536182,0.299898,0.820803
100,EMSR383_AOI04_09,3928050,19881,12861,1584220,0.506129,0.811819
